In [113]:
import hashlib
import secrets
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

# Short Private Keys

Instead of generating private keys using 256 random numbers we can instead seed a [CSPRNG](https://en.wikipedia.org/wiki/Cryptographically_secure_pseudorandom_number_generator) with one random number and then use the CSPRNG to generate the rest of the private key. This is a lot faster and uses less memory to store the private key.

### Private Key Size Comparison

Scheme | Private Key | Public Key | Total Size |
| --- | --- | --- | --- |
| Naive Lamport | $32 * 256 = 8192$ bytes | $32 * 256 = 8192$ bytes | $16384$ bytes |
| Short Private Key Lamport | $32 + 16 = 48$ bytes | $32 * 256 = 8192$ bytes | $8240$ bytes |

By using the *short private key* scheme we can reduce the total size of the private key by ~170x when compared to the *naive* scheme.

<!-- (48 / 8192) * 100 = 0.59 percent -->

In [114]:
####### CONSTANTS #######

# Provides roughly 128 bits of security (see Grover's algorithm)
security_parameter = 256 # 32 bytes

# Max message length that the private key can sign
max_message_length = 256 # 32 bytes

## AES CSPRNG

For this example implementation we will use the [AES](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard) cipher in [Counter Mode](https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation#Counter_(CTR)) as our [CSPRNG](https://en.wikipedia.org/wiki/Cryptographically_secure_pseudorandom_number_generator#Designs_based_on_cryptographic_primitives). 

In [115]:
CSPRNG_key = secrets.token_bytes(security_parameter // 8) # Private key is a random number (can be reused as long as nonce changes)
CSPRNG_nonce = secrets.token_bytes(16) # Nonce is a random number used only once

# Create a AES256 cipher object using counter mode
cipher = Cipher(algorithms.AES256(CSPRNG_key), modes.CTR(CSPRNG_nonce))

In [116]:
def hash_key_element(key_element: bytes) -> bytes:
    '''Hashes a single key element, producing a 256-bit digest'''

    return hashlib.sha3_256(key_element).digest()


def generate_public_key(private_key: list) -> list:
    '''Generates a public Lamport key from a private key'''

    # Generate public key by hashing each private key element
    return list(map(hash_key_element, private_key))


enc = cipher.encryptor()
zero_bytes = (0).to_bytes(length=32, byteorder='big')
private_key = [enc.update(zero_bytes) for _ in range(2 * max_message_length)]
public_key = generate_public_key(private_key)